# Dollar Auction Game Using OpenAI Agents
This notebook simulates Shubik's Dollar Auction game with two AI agents.

## Setup

In [1]:
#!pip install openai ipywidgets

In [2]:
from openai import OpenAI

client = OpenAI()
import os
import ipywidgets as widgets
from IPython.display import display

In [3]:
# Set your OpenAI API key. You can also set the OPENAI_API_KEY environment variable.openai.api_key = os.getenv("OPENAI_API_KEY")  # or replace with your key

## Define Characters

In [4]:
characters = {    "Rational": "A player who aims to maximize profit and avoid overbidding." ,
              "Escalator": "A player prone to escalation and emotional bidding." ,
              "Conservative": "A cautious player who bids minimally and may drop early." ,
              "Aggressive": "A player who bids high to intimidate the opponent."}

## Select Players

In [9]:
player1_widget = widgets.Dropdown(options=list(characters.keys()), description="Player 1:")
player2_widget = widgets.Dropdown(options=list(characters.keys()), description="Player 2:")
display(player1_widget, player2_widget)

Dropdown(description='Player 1:', options=('Rational', 'Escalator', 'Conservative', 'Aggressive'), value='Rati…

Dropdown(description='Player 2:', options=('Rational', 'Escalator', 'Conservative', 'Aggressive'), value='Rati…

## Define Dollar Auction Game Class

In [10]:
class DollarAuctionGame:
    def __init__(self, player1_role, player2_role, starting_bid=0.01, increment=0.01, max_bid=1.00):
        self.player_roles = {"player1": player1_role, "player2": player2_role}
        self.increment = increment
        self.max_bid = max_bid
        self.current_bid = starting_bid
        self.history = []
        self.bids = {"player1": 0.0, "player2": 0.0}

    def get_prompt(self, role, last_bid):
        role_desc = characters[role]
        prompt = f"""You are playing a Dollar Auction game as the '{role}' player.
Role description: {role_desc}
The current highest bid is ${last_bid:.2f}.
On your turn, either place a new bid of at least ${last_bid + self.increment:.2f} or say "stop" to drop out.
If you drop out, you pay your last bid. If both players stop, the highest bidder pays their bid, and the second-highest also pays theirs.
Return either a bid amount (e.g., '0.05') or 'stop'.
"""
        return prompt

    def play(self):
        last_bid = self.current_bid
        turn = 0
        while True:
            player = "player1" if turn % 2 == 0 else "player2"
            role = self.player_roles[player]
            prompt = self.get_prompt(role, last_bid)
            response = client.chat.completions.create(model="gpt-4",
            messages=[{"role": "system", "content": "You are playing a strategic bidding game."}, {"role": "user", "content": prompt}],
            temperature=0.7)
            content = response.choices[0].message.content.strip()
            self.history.append((role, content))
            if content.lower() == "stop":
                break
            try:
                bid_value = float(content.strip().lstrip('$'))
            except ValueError:
                break
            if bid_value < last_bid + self.increment:
                break
            last_bid = bid_value
            self.bids[player] = bid_value
            turn += 1
            if last_bid >= self.max_bid:
                break
        sorted_bids = sorted(self.bids.items(), key=lambda x: x[1], reverse=True)
        winner_key, winner_bid = sorted_bids[0]
        runner_key, runner_bid = sorted_bids[1]
        return {"history": self.history, "winner": self.player_roles[winner_key], "winner_bid": winner_bid, \
                "runner_up": self.player_roles[runner_key], "runner_bid": runner_bid}


## Run Simulation

In [11]:
game = DollarAuctionGame(player1_widget.value, player2_widget.value)

result = game.play()
print("Bidding History:")

for role, action in result["history"]:    
    print(f"{role}: {action}")
    print(f"\nWinner: {result['winner']} with bid ${result['winner_bid']:.2f}")
    print(f"Runner-up: {result['runner_up']} with bid ${result['runner_bid']:.2f}")

Bidding History:
Rational: 0.02

Winner: Rational with bid $0.05
Runner-up: Rational with bid $0.04
Rational: 0.03

Winner: Rational with bid $0.05
Runner-up: Rational with bid $0.04
Rational: 0.04

Winner: Rational with bid $0.05
Runner-up: Rational with bid $0.04
Rational: 0.05

Winner: Rational with bid $0.05
Runner-up: Rational with bid $0.04
Rational: 0.06

Winner: Rational with bid $0.05
Runner-up: Rational with bid $0.04
